In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras import layers
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# розмір картинки
IMG_SIZE = (75, 75)

# папка з датасетом у якій підпапки
# з фотографіями кожного символа 
directory = "/content/drive/MyDrive/Colab Notebooks/images/"


In [4]:
data_generator = ImageDataGenerator(validation_split=0.2, rescale=1.0/255)
 
# датасет для тренування
train_data = data_generator.flow_from_directory(directory,
                                               target_size=IMG_SIZE,
                                               class_mode='categorical',
                                               subset='training')

# датасет для тестування
test_data = data_generator.flow_from_directory(directory,
                                               target_size=IMG_SIZE,
                                               class_mode='categorical',
                                               subset='validation')

Found 286 images belonging to 29 classes.
Found 55 images belonging to 29 classes.


In [9]:
def createModel(image_shape = IMG_SIZE):
    '''
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
        tf.keras.model
    '''
    input_shape = image_shape + (3,)
    
    base_model = tf.keras.applications.InceptionV3(input_shape = input_shape,
                                               include_top = False,
                                               weights = "imagenet",
                                               classes = 29,
                                               classifier_activation = "softmax")
    # заморозимо шари, щоб їх не тренувати
    base_model.trainable = False

    # вхідний шар
    inputs = tf.keras.Input(shape = input_shape) 

    # для нормалізації інпута [-1;1]
    x = tf.keras.applications.inception_v3.preprocess_input(inputs) 

    x = base_model(x, training = False) 
    x = tfl.GlobalAveragePooling2D()(x) 
    x = tfl.Dropout(0.2)(x)
    outputs = tfl.Dense(29, activation = "softmax")(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [10]:
model = createModel(IMG_SIZE)

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [12]:
history = model.fit(train_data, validation_data = test_data, epochs = 10)

Epoch 1/10
9/9 [==============================] - 7s 420ms/step - loss: 3.3331 - accuracy: 0.1014 - val_loss: 2.0263 - val_accuracy: 0.6000
Epoch 2/10
9/9 [==============================] - 2s 262ms/step - loss: 2.2092 - accuracy: 0.3916 - val_loss: 1.3577 - val_accuracy: 0.7818
Epoch 3/10
9/9 [==============================] - 2s 267ms/step - loss: 1.6237 - accuracy: 0.6434 - val_loss: 0.9508 - val_accuracy: 0.8727
Epoch 4/10
9/9 [==============================] - 3s 294ms/step - loss: 1.2504 - accuracy: 0.7483 - val_loss: 0.6887 - val_accuracy: 0.9455
Epoch 5/10
9/9 [==============================] - 2s 265ms/step - loss: 0.9993 - accuracy: 0.7972 - val_loss: 0.5472 - val_accuracy: 0.9455
Epoch 6/10
9/9 [==============================] - 4s 453ms/step - loss: 0.8151 - accuracy: 0.8462 - val_loss: 0.4399 - val_accuracy: 0.9455
Epoch 7/10
9/9 [==============================] - 3s 353ms/step - loss: 0.6612 - accuracy: 0.8881 - val_loss: 0.3575 - val_accuracy: 0.9636
Epoch 8/10
9/9 [====